<a href="https://colab.research.google.com/github/sam-h-bean/imdb-goof/blob/master/lstm_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>